In [1]:
!python -V

Python 3.9.13


In [2]:
import pickle

In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import numpy as np

In [11]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [4]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db") 
mlflow.set_experiment("nyc-taxi-experiment3")

2025/06/17 17:29:46 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/06/17 17:29:46 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


<Experiment: artifact_location='/workspaces/MLops/02-experiment-tracking/mlruns/5', creation_time=1749895298338, experiment_id='5', last_update_time=1749895298338, lifecycle_stage='active', name='nyc-taxi-experiment3', tags={}>

In [5]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration']=df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    
    df = df[((df.duration>=1) & (df.duration <=60))]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [6]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')

In [7]:
len(df_train), len(df_val)

(73908, 61921)

In [8]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [12]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [13]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [14]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

rmse=np.sqrt(mean_squared_error(y_val, y_pred))
print(f'RMSE: {rmse:.3f}')

RMSE: 7.759


In [13]:
with mlflow.start_run():
    
    mlflow.set_tag("developer", "Billol")
    
    mlflow.log_param("train-data-path", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet")
    mlflow.log_param("val-data-path", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet")
    
    alpha = 0.01
    mlflow.log_param("alpha", alpha)
    lr = LinearRegression()
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse=np.sqrt(mean_squared_error(y_val, y_pred))
    print(f'RMSE: {rmse:.3f}')
    mlflow.log_metric("rmse", rmse)
    
    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

RMSE: 7.759


In [14]:
import os
print(os.getcwd())  # Shows current working directory

/workspaces/MLops/02-experiment-tracking


In [15]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [16]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [17]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [18]:
def objective(params):
    with mlflow.start_run():
        
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        
        y_pred = booster.predict(valid)
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        mlflow.log_metric("rmse", rmse)
        
    return {'loss': rmse, 'status': STATUS_OK}

In [19]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=10,
    trials=Trials()
)

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [16:28:23] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:10.71635                          
[1]	validation-rmse:9.59436                           
[2]	validation-rmse:8.76615                           
[3]	validation-rmse:8.16221                           
[4]	validation-rmse:7.72942                           
[5]	validation-rmse:7.42169                           
[6]	validation-rmse:7.20229                           
[7]	validation-rmse:7.04637                           
[8]	validation-rmse:6.93540                           
[9]	validation-rmse:6.85431                           
[10]	validation-rmse:6.79524                          
[11]	validation-rmse:6.75229                          
[12]	validation-rmse:6.71946                          
[13]	validation-rmse:6.69485                          
[14]	validation-rmse:6.67615                          
[15]	validation-rmse:6.66201                          
[16]	validation-rmse:6.64913                          
[17]	validation-rmse:6.63983                          
[18]	valid

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [16:28:38] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:11.38287                                                  
[1]	validation-rmse:10.66672                                                  
[2]	validation-rmse:10.05198                                                  
[3]	validation-rmse:9.52659                                                   
[4]	validation-rmse:9.07971                                                   
[5]	validation-rmse:8.70163                                                   
[6]	validation-rmse:8.38219                                                   
[7]	validation-rmse:8.11355                                                   
[8]	validation-rmse:7.88839                                                   
[9]	validation-rmse:7.69968                                                   
[10]	validation-rmse:7.54115                                                  
[11]	validation-rmse:7.40971                                                  
[12]	validation-rmse:7.29947                        

...

[0]	validation-rmse:6.97623                                                    
[1]	validation-rmse:6.69656                                                    
[2]	validation-rmse:6.64402                                                    
[3]	validation-rmse:6.62586                                                    
[4]	validation-rmse:6.61135                                                    
[5]	validation-rmse:6.60555                                                    
[6]	validation-rmse:6.59927                                                    
[7]	validation-rmse:6.59313                                                    
[8]	validation-rmse:6.59002                                                    
[9]	validation-rmse:6.58968                                                    
[10]	validation-rmse:6.58632                                                   
[11]	validation-rmse:6.58479                                                   
[12]	validation-rmse:6.58325            

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [16:37:04] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:10.09412                                                   
[1]	validation-rmse:8.74082                                                    
[2]	validation-rmse:7.90448                                                    
[3]	validation-rmse:7.40118                                                    
[4]	validation-rmse:7.10091                                                    
[5]	validation-rmse:6.91712                                                    
[6]	validation-rmse:6.80477                                                    
[7]	validation-rmse:6.73150                                                    
[8]	validation-rmse:6.68330                                                    
[9]	validation-rmse:6.65106                                                    
[10]	validation-rmse:6.63112                                                   
[11]	validation-rmse:6.61697                                                   
[12]	validation-rmse:6.60489            

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [16:37:25] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:10.88398                                                   
[1]	validation-rmse:9.84994                                                    
[2]	validation-rmse:9.05345                                                    
[3]	validation-rmse:8.44961                                                    
[4]	validation-rmse:7.99495                                                    
[5]	validation-rmse:7.65458                                                    
[6]	validation-rmse:7.40263                                                    
[7]	validation-rmse:7.21547                                                    
[8]	validation-rmse:7.07714                                                    
[9]	validation-rmse:6.97400                                                    
[10]	validation-rmse:6.89707                                                   
[11]	validation-rmse:6.83856                                                   
[12]	validation-rmse:6.79259            